Final Code

In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager



def get_url(search):
    template='https://www.amazon.com/s?k={}&ref=nav_bb_sb'
    search=search.replace(' ','+')

    url=template.format(search)
    #Adding page='any num' so we can get details from each page
    url+='&page={}'
    return url

def myfunction_to_extract_records(item):

    atag=item.h2.a
    #To get the Product Name
    description = atag.text.strip()
    url = 'https://www.amazon.com'+atag.get('href')
    #Error Handling in case we have products with no price
    try:
        #To get the product price
        price_parent=item.find('span','a-price')
        price=price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    #Error Handling in case we have products with no rating or review count
    try:
        #To get the product rating and review count
        rating=item.i.text
        review_count=item.find('span','a-size-base').text
    except AttributeError:
        rating=''
        review_count=''

    #Stroring final result
    result = (description,price,rating,review_count,url)

    return result

def main(search):

    #Opening the driver with crome
    driver = webdriver.Chrome(ChromeDriverManager().install())

    records=[]
    url=get_url(search)

    #Iterating through the pages
    inpu2=int(input("How many pages?"))
    for page in range(1,inpu2+1):
        driver.get(url.format(page))
        #using BeautifulSoup for parsing HTML
        soup = BeautifulSoup(driver.page_source,'html.parser')

        #finding all the products on Amazon page
        results=soup.find_all('div',{'data-component-type':'s-search-result'})

        #iterating through each product and fetching the information
        for item in results:
            record=myfunction_to_extract_records(item)
            if record:
                records.append(record)
    #closing the driver after use
    driver.close()
    #Storing all the data in CSV file
    with open('results.csv','w',newline='',encoding='utf-8') as var:
        writer=csv.writer(var)
        writer.writerow(['Description','Price','Rating', 'Review Count', 'URL'])
        writer.writerows(records)

In [2]:
inpu=input("Enter the product")
main(inpu)

C:\Users\bhaga\AppData\Local\Temp\ipykernel_5972\4212916001.py:48: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
